In [114]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
from sklearn.preprocessing import StandardScaler

In [115]:
DATA_SUB_DIR = '../data/'

# Load data
df_transactions = pd.read_csv(DATA_SUB_DIR + 'transactions.csv', sep=',')
df_mcc_codes = pd.read_csv(DATA_SUB_DIR +'mcc_codes.csv', sep=';')
df_trans_types = pd.read_csv(DATA_SUB_DIR + 'trans_types.csv', sep=';')
df_train = pd.read_csv(DATA_SUB_DIR + 'train.csv', sep=',', index_col='client_id').drop('Unnamed: 0', axis=1)

In [116]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3563529 entries, 0 to 3563528
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   client_id   object 
 1   trans_time  object 
 2   mcc_code    int64  
 3   trans_type  int64  
 4   amount      float64
 5   term_id     object 
 6   trans_city  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 190.3+ MB


In [117]:
# Define a range of intervals to evaluate
intervals = [(-x*3_000, x*3_000) for x in range(13, 20)]

# Function to evaluate outliers for a given interval
def evaluate_outliers(df, interval):
    lower_bound, upper_bound = interval
    outliers = df[(df['amount'] < lower_bound) | (df['amount'] > upper_bound)]
    num_outliers = len(outliers)
    percentage_outliers = num_outliers / len(df) * 100
    return num_outliers, percentage_outliers

# Evaluate each interval
results = []
for interval in intervals:
    num_outliers, percentage_outliers = evaluate_outliers(df_transactions, interval)
    results.append({
        'interval': interval,
        'num_outliers': num_outliers,
        'percentage_outliers': round(percentage_outliers, 3)
    })

# Convert results to DataFrame for easy visualization
df_results = pd.DataFrame(results)

# Print results
print(df_results)
# clar memory fom new temporary variables
del num_outliers, percentage_outliers, results, intervals, df_results, interval

          interval  num_outliers  percentage_outliers
0  (-39000, 39000)         32706                0.918
1  (-42000, 42000)         30529                0.857
2  (-45000, 45000)         27808                0.780
3  (-48000, 48000)         26442                0.742
4  (-51000, 51000)         24398                0.685
5  (-54000, 54000)         23719                0.666
6  (-57000, 57000)         22617                0.635


it looks like the interval (-50_000, 50_000) is a good compromise for identifying outliers in the 'amount' column. 

In [118]:
# so drop the ouliers outs of this interval from the transactions dataframe.
df_transactions = df_transactions[(df_transactions['amount'] >= -50_000) & (df_transactions['amount'] <= 50_000)]

lets format the time of transaction

In [119]:

df_transactions['day'] = df_transactions['trans_time'].str.split().apply(lambda x: int(x[0]) % 7)
df_transactions['hour'] = df_transactions['trans_time'].apply(lambda x: int(x.split()[1].split(':')[0]))
df_transactions['is_night'] = df_transactions['hour'].apply(lambda x: 1 if x >= 22 or x <= 6 else 0)
# also drop term_id since it's not informative for this analysis
df_transactions.drop('term_id', axis=1, inplace=True)

In [120]:
df_transactions = pd.get_dummies(df_transactions, columns=['mcc_code'])
df_transactions = pd.get_dummies(df_transactions, columns=['trans_type'])

In [121]:
# Scaling and Normalization
scaler = StandardScaler()
numerical_features = ['amount']
df_transactions[numerical_features] = scaler.fit_transform(df_transactions[numerical_features])

# Correlation Analysis: Remove highly correlated features to reduce multicollinearity.

corr_matrix = df_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
df_train.drop(to_drop, axis=1, inplace=True)

/var/folders/mk/pn_xz2mj3n7_55rszn3mpjb00000gn/T/ipykernel_21786/3528088338.py:9: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

NameError: name 'X' is not defined